<p style="text-align: center;"> <span style="color:firebrick"> <font size="5"> <b> USC Marshall School of Business </b> </font> </p> </span> 

<p style="text-align: center;"> <b> <font font size="5"> DSO 560 - Final Project </p> </b></font>

<p style="text-align: center;"> <b> Spring 2021 </b> </p>

## <span style="color:black"> <font size="3">By: Ningchuan Peng</span>

In [5]:
# import packages
import re
import numpy as np
import pandas as pd
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from scipy.spatial.distance import cosine
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

[nltk_data] Error loading wordnet: <urlopen error [Errno 61]
[nltk_data]     Connection refused>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno 61] Connection refused>


In [6]:
# define the necessary functions

# lowercase
def lowercase(title):
    if isinstance(title, str):
        title = title.lower()
    return title

# count
def count(title, word):
    if isinstance(title, str):
        count = re.findall(word, title)
    return len(count)

# replace words
def replace(title, word, reword):
    count = 0
    if isinstance(title, str):
        title = re.sub(word, reword, title)
    return title

# remove stopwords
english_stopwords = set(stopwords.words("english"))
english_stopwords
def remove_st(title):
    if isinstance(title, str):
        tokens = nltk.word_tokenize(title)
        filtered_tokens = []
        for token in tokens:
            if token in english_stopwords:
                continue
            filtered_tokens.append(token.lower())
        return " ".join(filtered_tokens)
    
# lemmentization
def lemm(title):
    if isinstance(title, str):
        tokens = nltk.word_tokenize(title)
        filtered_tokens = []
        for token in tokens:
            token = WordNetLemmatizer().lemmatize(token)
            filtered_tokens.append(token.lower())
        return " ".join(filtered_tokens)

## Question 1

Build an NLP classification model to predict which brand a new product should be assigned.

In [9]:
# read the dataset
brand_df = pd.read_csv('behold_brands.csv')
product_df = pd.read_excel('Behold+product+data+04262021.xlsx')

# change nan to space
product_df.fillna(' ', inplace = True)
product_df.head(3)

,product_id,brand,brand_category,name,details,created_at,brand_canonical_url,description,brand_description,brand_name,product_active
0,01EX0PN4J9WRNZH5F93YEX6QAF,Two,Unknown,Khadi Stripe Shirt-our signature shirt,,2021-01-27 01:17:19.305 UTC,https://two-nyc.myshopify.com/products/white-k...,Our signature khadi shirt\navailable in black ...,Our signature khadi shirt\n\navailable in blac...,Khadi Stripe Shirt-our signature shirt,True
1,01F0C4SKZV6YXS3265JMC39NXW,Collina Strada,Unknown,RUFFLE MARKET DRESS LOOPY PINK SISTINE TOMATO,,2021-03-09 18:43:10.457 UTC,https://collina-strada-2.myshopify.com/product...,Mid-length dress with ruffles and adjustable s...,Mid-length dress with ruffles and adjustable s...,RUFFLE MARKET DRESS LOOPY PINK SISTINE TOMATO,True
2,01EY4Y1BW8VZW51BWG5VZY82XW,Cariuma,Unknown,IBI Slip On Raw Red Knit Sneaker Women,,2021-02-10 02:58:59.591 UTC,https://cariuma.myshopify.com/products/ibi-sli...,IBI Slip On Raw Red Knit Sneaker Women,IBI Slip On Raw Red Knit Sneaker Women,IBI Slip On Raw Red Knit Sneaker Women,False


In [4]:
# create a column with detail and description
product_df['de_de'] = product_df['details'].astype(str) + ' ' + product_df['description'].astype(str)

In [5]:
# change to lowercase
product_df['de_de'] = product_df['de_de'].apply(lowercase)

# replace 
product_df['de_de'] = product_df['de_de'].apply(lambda x: replace(x, r'\d+|\d+[a-z]+', ''))
product_df['de_de'] = product_df['de_de'].apply(lambda x: replace(x, r'½ï', ''))

# remove stopwords    
product_df['de_de'] = product_df['de_de'].apply(remove_st)

# use lemmatization
product_df['de_de'] = product_df['de_de'].apply(lemm)

In [6]:
# create a brand mapping: except the first 30 brands, others are classified as "Other brand"
L1 = list(product_df['brand'].value_counts()[:30].index)
d1 = dict.fromkeys(L1)
for key in d1.keys():
    d1[key] = key

L2 = list(product_df['brand'].value_counts()[30:].index)
d2 = dict.fromkeys(L2, 'Other brand')

d = {**d1, **d2}

brand_pre = product_df['brand'].map(d)
brand_pre.head()

0       Other brand
1    Collina Strada
2       Other brand
3       Other brand
4       Other brand
Name: brand, dtype: object

In [7]:
# transform to TF-IDF vector
vectorizer = TfidfVectorizer(min_df=100, max_features = 300, ngram_range=(1,1))
test = vectorizer.fit_transform(product_df['de_de'])
product_tfidf = pd.DataFrame(test.toarray(), columns = vectorizer.get_feature_names())

# change the BRAND column to number
product_tfidf['BRAND'] = pd.factorize(brand_pre)[0] + 1
product_tfidf.head()

,add,adjustable,also,angeles,ankle,approximately,around,artisan,available,back,...,work,worn,woven,wrap,xs,yet,york,zip,zipper,BRAND
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.195196,0.0,...,0.0,0.0,0.214807,0.0,0.0,0.0,0.00000,0.0,0.000000,1
1,0.0,0.248763,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.30119,0.0,0.244957,2
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.000000,1
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.000000,1
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.000000,1


In [8]:
# kfolds and feature result split
kfolds = StratifiedKFold(n_splits = 5, random_state = 0, shuffle = True)

feature_col = list(product_tfidf.columns)
feature_col.remove('BRAND')
X_train_CV = product_tfidf[feature_col].values

In [9]:
# LoR classification CV
cross_val_score(LogisticRegression(max_iter=3000), X_train_CV,  product_tfidf['BRAND'], cv = kfolds)

array([0.85282373, 0.85812077, 0.85600196, 0.85078641, 0.85461658])

In [10]:
# Random Forest classification CV (RF is better than LoR)
cross_val_score(RandomForestClassifier(random_state=0), X_train_CV,  product_tfidf['BRAND'], cv = kfolds)

array([0.90098606, 0.90391981, 0.90432728, 0.90318637, 0.90343085])

In [11]:
# train test split
X, y = product_tfidf.iloc[:, 0:300].values, product_tfidf.iloc[:,300].values 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 0, stratify = y)

# RF with 10% test 
rf_fit = RandomForestClassifier(random_state=0).fit(X_train, y_train)
print('Classifcation error on test set:', 1 - rf_fit.score(X_test, y_test))

Classifcation error on test set: 0.09615384615384615


In [12]:
# final outcome
rf_fit.predict(X_test)

array([14, 11,  7, ...,  1,  1,  1])

## Question 2

Create a brand recommender algorithm that would recommend an outfit given a customer’s search query. 

In [7]:
# read the data
outfit_df = pd.read_csv('outfit_combinations.csv')
outfit_df.head(3)

,outfit_id,product_id,outfit_item_type,brand,product_full_name
0,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2P5H24WK0HTK4R0A1,bottom,Eileen Fisher,Slim Knit Skirt
1,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2PEPWFTT7RMP5AA1T,top,Eileen Fisher,Rib Mock Neck Tank
2,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2S5T9W793F4CY41HE,accessory1,kate spade new york,medium margaux leather satchel


### Outfit TF-IDF Grouping method

In [10]:
# combine three descriptive columns together
outfit_de = outfit_df.merge(product_df[['product_id','details','description']], 
                            on=['product_id', 'product_id'], 
                            how="left")
outfit_de.fillna(' ', inplace = True)
outfit_de['de_de'] = outfit_de['product_full_name'] + ' ' + outfit_de['details'] + ' ' + outfit_de['description']+ ' '

# change the data to pivotal form
outfit_pivot = outfit_de.groupby('outfit_id').agg({'de_de':'sum'})
outfit_pivot.head(3)

,de_de
outfit_id,
01DDBHC62ES5K80P0KYJ56AM2T,Slim Knit Skirt Rib Mock Neck Tank med...
01DMHCX50CFX5YNG99F3Y65GQW,Slim Knit Skirt Rib Mock Neck Tank Pen...
01DMHRX35M2DPVYVQ1PNER4S4B,Chemelle Midi Dress Penelope Mid Cap Toe P...


In [11]:
# change to lowercase
outfit_pivot['de_de'] = outfit_pivot['de_de'].apply(lowercase)

# replace 
outfit_pivot['de_de'] = outfit_pivot['de_de'].apply(lambda x: replace(x, r'\d+|\d+[a-z]+', ''))

# remove stopwords    
outfit_pivot['de_de'] = outfit_pivot['de_de'].apply(remove_st)

# use lemmatization
outfit_pivot['de_de'] = outfit_pivot['de_de'].apply(lemm)

In [12]:
# transform to TF-IDF vector
vectorizer = TfidfVectorizer(min_df=20, ngram_range=(1,1))
test = vectorizer.fit_transform(outfit_pivot['de_de'])
pd.DataFrame(test.toarray(), columns = vectorizer.get_feature_names()).head()

,ab,abstract,accent,accentuate,accessory,acetate,achieve,across,acrylic,act,...,xxl,xxs,yarn,year,yet,yoga,yoke,zebra,zip,zipped
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# define the search function
def search(query):
    # change the query to TF-IDF vector
    query = lemm(remove_st(replace(lowercase(query), r'\d+|\d+[a-z]+', '')))
    query_tfidf = vectorizer.transform([query]).toarray()    
    
    # find the lowest cosine distance
    min_count = np.inf
    for i in range(test.toarray().shape[0]):
        dist = cosine(test.toarray()[i], query_tfidf)
        if dist < min_count:
            min_count = dist
            min_i = i

    # find the best outfit
    final_outfit = outfit_df[outfit_df['outfit_id']==outfit_pivot.index[min_i]]

    # print the result
    for i in range(final_outfit.shape[0]):
        out_type = final_outfit.iloc[i, 2].capitalize()
        out_brand = final_outfit.iloc[i, 3]
        out_prod = final_outfit.iloc[i, 4]
        out_id = final_outfit.iloc[i, 1]
        print(f'{out_type}: {out_brand} {out_prod} ({out_id})')
        print(min_count)

In [14]:
# for test
test_query = input('Please input your search words: ')
if test_query == '':
    test_query = 'slim fitting, straight leg pant with a center back zipper and slightly cropped leg'
    print('Example: slim fitting, straight leg pant with a center back zipper and slightly cropped leg')
search(test_query)

Please input your search words: 
Example: slim fitting, straight leg pant with a center back zipper and slightly cropped leg
Top: J.Crew Balloon sleeve cable knit cardigan (01DPC6Z5K3ZB8KZ0M6YW7Z4JX8)
0.6872990508520548
Shoe: J.Crew Sadie boots in suede (01DPEEZGRJYX4N4WE78GPM0B2G)
0.6872990508520548
Bottom: Reformation Liza High Straight Crop (01DPKMXBJEMQWQWAZHGJK1RS8Z)
0.6872990508520548
